In [33]:
import pandas as pd 
import plotly.express as px
import marimo as mo

In [19]:
def read_csv_file(file_path):
    return pd.read_csv(file_path)

file_paths = [
    'data/olist_order_items_dataset.csv',
    'data/olist_order_reviews_dataset.csv',
    'data/olist_orders_dataset.csv',
    'data/WA_Fn-UseC_-HR-Employee-Attrition.csv'
]


dataframes = [read_csv_file(file_path) for file_path in file_paths]

# Unpack the dataframes
df_order_items, df_order_reviews, df_orders, df_people = dataframes


In [20]:
columns = [df_order_items.columns, df_order_reviews.columns,df_orders.columns, df_people.columns]
print(columns)

[Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object'), Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object'), Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object'), Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRati

In [21]:
#data cleaning
orderCat = df_orders['order_status'].unique().tolist()
orderCat

['delivered',
 'invoiced',
 'shipped',
 'processing',
 'unavailable',
 'canceled',
 'created',
 'approved']

In [22]:
# Calculating the product of 'order_item_id' and 'price' for each item
trxProduct = df_order_items['order_item_id'].mul(df_order_items['price'])

# Adding the product to df_order_items as a column
df_order_items2 = df_order_items.assign(trxProduct=trxProduct)

In [23]:
#visualization of distribution
priceDistribution = px.histogram(df_order_items2, x='trxProduct', 
                                labels={'trxProduct': 'Total Transaction (price x quantity)'}, 
                                title='Transaction Value Distribution'
                                )

priceDistribution.show()

In [24]:
# Calculating the median of the resulting values
# Average order value
median_value = trxProduct.median()
median_value

84.495

In [26]:
# Fulfillment Metrics
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])

# Order cycle time
# Calculate the difference between two datetime columns
orderCycleTime = df_orders['order_delivered_customer_date'].sub(df_orders['order_approved_at'])
orderCycleTime

0        8 days 10:17:58
1       12 days 12:03:18
2        9 days 09:11:06
3       13 days 04:42:43
4        2 days 19:56:33
              ...       
99436    8 days 05:13:56
99437   22 days 04:27:19
99438   24 days 20:20:01
99439   17 days 01:56:33
99440    7 days 01:48:02
Length: 99441, dtype: timedelta64[ns]

In [27]:

# Assuming orderCycleTime is a pandas Series containing the order cycle time in days

# Create a DataFrame with the order cycle time
df_order_cycle_time = pd.DataFrame(orderCycleTime, columns=['order_cycle_time'])

# Create a histogram with the DataFrame
fig = px.histogram(data_frame=df_order_cycle_time, x='order_cycle_time', nbins=30, title='Order Cycle Time Histogram')
fig.update_xaxes(title_text='Days')
fig.update_yaxes(title_text='Frequency')

# Display the histogram
fig.show()


In [28]:
# Calculating the median of the resulting values
# Order cycle time
octMedian = orderCycleTime.dt.days.median()
octMedian

9.0

In [29]:
# Average rating
scoreCounting = df_order_reviews['review_score'].value_counts()


# Convert the value counts series to a DataFrame and reset the index
scoreCounting_df = scoreCounting.reset_index(name='count')

# Sort the DataFrame by the review_score column
scoreCounting_df = scoreCounting_df.sort_values(by='count', ascending=False)

print(scoreCounting_df)

# Create bar chart
bar_chart = px.bar(scoreCounting_df, x='review_score', y='count', 
                   labels={'review_score': 'Review Score', 'count': 'Count'},
                   title='Distribution of Review Scores')


# Show the bar chart
bar_chart.show()


   review_score  count
0             5  57328
1             4  19142
2             1  11424
3             3   8179
4             2   3151


In [30]:
# Calculating the median of the resulting values
# review score
df_order_reviews['review_score'].median()

5.0

In [31]:
#visualization of job satisfaction distribution
satisfactionDistribution = px.histogram(df_people, x='JobSatisfaction', 
                                labels={'JobSatisfaction': 'Job Satisfaction'}, 
                                title='Job Satisfaction Distribution'
                                )

satisfactionDistribution.show()

In [32]:
df_people['JobSatisfaction'].median()

3.0

In [39]:
mo.hstack([
    mo.stat(value="100.54", label="Open price", caption="2.4", direction="increase", bordered=True),
    mo.stat(value="100.54", label="Close price", caption="2.4", direction="decrease", bordered=True),
])


# References
* https://beprofit.co/a/blog/essential-shipping-performance-metrics-for-e-commerce-stores